<a href="https://colab.research.google.com/github/Pabloprogramador23/CREWAI---/blob/main/TranscrevEmMp3LLMWhisper1Egpt4mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalação das bibliotecas necessárias
!pip install pydub nltk
!pip install openai==0.28
!sudo apt-get update && sudo apt-get install -y ffmpeg
!pip install python-dotenv

In [18]:
import os
from google.colab import files
import openai
import nltk
from nltk.tokenize import sent_tokenize
from pydub import AudioSegment
from pydub.silence import split_on_silence
from dotenv import load_dotenv


In [19]:

nltk.download('punkt')

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

# Configuração da API OpenAI
openai.api_key = os.getenv("OPENAI_API_KEY")

def time_format(milliseconds):
    seconds = milliseconds / 1000
    return f"{int(seconds // 60):02d}:{int(seconds % 60):02d}"

def transcribe_audio_whisper(audio_file_path):
    with open(audio_file_path, "rb") as audio_file:
        transcript = openai.Audio.transcribe("whisper-1", audio_file)
    return transcript["text"]

def improve_transcript(text):
    """Melhora a transcrição utilizando a API do OpenAI com um prompt refinado."""
    completion = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": (
                    "Você é um especialista em transcrição de áudio e revisão gramatical da língua portuguesa brasileira. "
                    "Sua tarefa é revisar e melhorar transcrições de áudio, corrigindo erros gramaticais e de sintaxe, ajustando a pontuação e a formatação para garantir clareza e fluidez. "
                    "Além disso, você deve inferir o contexto e sugerir palavras ou frases mais apropriadas em partes do texto que possam estar incompletas ou mal transcritas devido à baixa qualidade do áudio ou presença de ruídos de fundo. "
                    "Também deve corrigir palavras que façam sentido foneticamente, mas que não sejam semanticamente adequadas ao contexto. "
                    "Use o contexto do áudio original e faça ajustes conforme necessário para que o texto final seja claro, coeso e natural."
                )
            },
            {
                "role": "user",
                "content": (
                    f"Revise e melhore o seguinte texto, garantindo que ele esteja gramaticalmente correto, bem pontuado, "
                    f"formatado em parágrafos apropriados e sem inconsistências. Considere o contexto do áudio original, incluindo possíveis erros de transcrição devidos a áudio de baixa qualidade ou trechos inaudíveis: {text}"
                )
            }
        ]
    )
    return completion.choices[0].message.content

def process_audio(audio_path, chunk_path, transcript_path):
    sound = AudioSegment.from_mp3(audio_path)
    chunks = split_on_silence(sound, min_silence_len=500, silence_thresh=sound.dBFS-14, keep_silence=500)
    full_transcript = ""
    current_time = 0
    speaker = 1
    for i, chunk in enumerate(chunks):
        chunk_name = f"{chunk_path}/chunk{i}.wav"
        chunk.export(chunk_name, format="wav")
        transcript = transcribe_audio_whisper(chunk_name)
        sentences = sent_tokenize(transcript)
        for sentence in sentences:
            full_transcript += f"[{time_format(current_time)}] Falante {speaker}: {sentence}\n"
            speaker = 2 if speaker == 1 else 1
        current_time += len(chunk)
    improved_transcript = improve_transcript(full_transcript)
    with open(transcript_path, "w", encoding="utf-8") as f:
        f.write(improved_transcript)

def main():
    print("Por favor, faça o upload do arquivo MP3.")
    uploaded = files.upload()
    mp3_path = next(iter(uploaded))
    chunk_path = "audio_chunks"
    transcript_path = "transcricao.txt"
    if not os.path.exists(chunk_path):
        os.mkdir(chunk_path)
    process_audio(mp3_path, chunk_path, transcript_path)
    print(f"Transcrição completa! O resultado está em {transcript_path}")
    files.download(transcript_path)

if __name__ == "__main__":
    main()

Por favor, faça o upload do arquivo MP3.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Saving ALERTA 1.mp3 to ALERTA 1 (5).mp3
Transcrição completa! O resultado está em transcricao.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>